In [32]:
%load_ext line_profiler
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import numpy as np
import qcgpu
from projectq import MainEngine
import projectq.ops as ops
from projectq.backends import Simulator
import sys
import time

set_matplotlib_formats('png', 'pdf')
plt.style.use('ggplot')

In [27]:
def plot_qcgpu_state(state):
    labels = map(
        lambda i: np.binary_repr(i, state.num_qubits), 
        range(0, 2**state.num_qubits)
    )

    y_pos = np.arange(len(labels))
    performance = list(state.probabilities())
 
    plt.bar(y_pos, performance, align='center')
    plt.xticks(y_pos, labels)
    plt.ylabel('Amplitude')
    plt.title('Probability Amplitudes')

    plt.show()

In [28]:
def bench_qcgpu(n, depth):
    state = qcgpu.State(n)

    h = qcgpu.gate.h()
    x = qcgpu.gate.x()
    sqrt_x = qcgpu.gate.sqrt_x()

    print('started')
    start = time.time()

    for level in range(depth):
        for q in range(n):
    
            state.apply_gate(h, q)
            state.apply_gate(sqrt_x, q)

            if q != 0:
                state.apply_controlled_gate(x, q, 0)
        
    runtime = time.time() - start
    print('ended: ', runtime)
    return runtime
    
def bench_projectq(n, depth):
    eng = MainEngine(backend=Simulator(gate_fusion=True), engine_list=[])
    qbits = eng.allocate_qureg(n)

    print('started')
    start = time.time()

    for level in range(depth):
        for q in qbits:
            ops.H | q
            ops.SqrtX | q
            if q != qbits[0]:
                ops.CNOT | (q, qbits[0])

    runtime = time.time() - start
    print('ended: ', runtime)

    for q in qbits:
        ops.Measure | q
    eng.flush()
    return runtime


In [29]:
# h = qcgpu.gate.h()
# x = qcgpu.gate.x()
# s = qcgpu.gate.s()
# t = qcgpu.gate.t()
# y = qcgpu.gate.y()
# z = qcgpu.gate.z()

# # Random Circuit
# state = qcgpu.State(5)

# state.apply_gate(h, 0)
# state.apply_gate(t, 0)
# state.apply_gate(y, 0)
# state.apply_gate(z, 0)
# state.apply_gate(h, 0)

# state.num_qubits

In [35]:
%lprun -f bench_qcgpu bench_qcgpu(26,5)

started
('ended: ', 0.2988259792327881)


In [31]:
# bench_projectq(26,5)